In [54]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [55]:
excel_file = 'Invoice.xlsx'
sheet_name = '1'
columns = ['Invoice Date', 'Vendor ID', 'Invoice Amount']

df = pd.read_excel(excel_file, sheet_name=sheet_name, usecols=columns)

df = df.dropna()

print(df)

    Invoice Date  Vendor ID  Invoice Amount
0     2022-06-15      65670         1732.67
1     2022-08-14      65670         2045.03
2     2022-10-13      65670         1398.03
3     2022-12-12      65670         1316.33
4     2023-02-10      65670         1755.29
..           ...        ...             ...
858   2023-11-12      65769         1308.83
859   2023-12-12      65769         1469.97
860   2024-01-11      65769         1956.70
861   2024-02-10      65769         2221.93
862   2024-03-11      65769         1410.39

[863 rows x 3 columns]


In [56]:
# Preprocess dates

def date_to_num(date_obj):
    base_date = np.datetime64('2000-01-01')
    delta_days = (date_obj - base_date).days
    return delta_days + 1

def date_from_num(days_count):
    base_date = np.datetime64('2000-01-01')
    target_date = base_date + np.timedelta64(days_count - 1, 'D')
    formatted_date = target_date.astype(object).strftime('%d-%b-%Y')
    return formatted_date

In [57]:
df['Invoice Date'] = df['Invoice Date'].apply(date_to_num)

print(df)

     Invoice Date  Vendor ID  Invoice Amount
0            8202      65670         1732.67
1            8262      65670         2045.03
2            8322      65670         1398.03
3            8382      65670         1316.33
4            8442      65670         1755.29
..            ...        ...             ...
858          8717      65769         1308.83
859          8747      65769         1469.97
860          8777      65769         1956.70
861          8807      65769         2221.93
862          8837      65769         1410.39

[863 rows x 3 columns]


In [58]:
print(df.head())

   Invoice Date  Vendor ID  Invoice Amount
0          8202      65670         1732.67
1          8262      65670         2045.03
2          8322      65670         1398.03
3          8382      65670         1316.33
4          8442      65670         1755.29


In [153]:
# Find Potential Vendor ID's

# Define the start and end dates for April 2024
start_date = np.datetime64('2024-04-01')
end_date = np.datetime64('2024-04-30')

potential_vendors = set()

# Iterate through each day in April 2024
current_date = start_date
while current_date <= end_date:
    day_count = (current_date - np.datetime64('2000-01-01')).astype(int) + 1
    
    # Filter the dataframe for each date offset
    filtered_df1 = df[df['Invoice Date'] == day_count-15]
    filtered_df2 = df[df['Invoice Date'] == day_count-30]
    filtered_df3 = df[df['Invoice Date'] == day_count-45]
    filtered_df4 = df[df['Invoice Date'] == day_count-60]
    
    # Add unique Vendor IDs to potential_vendors set
    # Iterate through filtered DataFrames and add tuples to the set
    for filtered_df in [filtered_df1, filtered_df2, filtered_df3, filtered_df4]:
        for index, row in filtered_df.iterrows():
            vendor_date_tuple = (day_count, row['Vendor ID'])
            potential_vendors.add(vendor_date_tuple)
    # potential_vendors.update(set(filtered_df1['Vendor ID']))
    # potential_vendors.update(set(filtered_df2['Vendor ID']))
    # potential_vendors.update(set(filtered_df3['Vendor ID']))
    # potential_vendors.update(set(filtered_df4['Vendor ID']))
    
    current_date += np.timedelta64(1, 'D')

In [154]:
print(potential_vendors)

vendors_df = pd.DataFrame(list(potential_vendors), columns=columns[:-1])

final_vendors_df = vendors_df.groupby('Vendor ID')['Invoice Date'].min().reset_index()

print(final_vendors_df)

{(8866, 65723.0), (8879, 65720.0), (8885, 65736.0), (8865, 65712.0), (8882, 65728.0), (8869, 65703.0), (8869, 65767.0), (8867, 65684.0), (8884, 65764.0), (8867, 65693.0), (8865, 65760.0), (8868, 65722.0), (8883, 65695.0), (8880, 65760.0), (8881, 65768.0), (8881, 65731.0), (8882, 65769.0), (8882, 65714.0), (8883, 65752.0), (8861, 65694.0), (8878, 65710.0), (8864, 65720.0), (8860, 65747.0), (8870, 65733.0), (8875, 65747.0), (8867, 65679.0), (8885, 65733.0), (8879, 65735.0), (8869, 65764.0), (8865, 65675.0), (8870, 65726.0), (8867, 65681.0), (8866, 65768.0), (8884, 65761.0), (8880, 65675.0), (8885, 65726.0), (8866, 65731.0), (8863, 65677.0), (8882, 65684.0), (8883, 65722.0), (8885, 65737.0), (8881, 65685.0), (8865, 65716.0), (8864, 65735.0), (8868, 65696.0), (8869, 65743.0), (8869, 65761.0), (8865, 65718.0), (8882, 65679.0), (8865, 65727.0), (8867, 65724.0), (8880, 65718.0), (8880, 65727.0), (8881, 65744.0), (8881, 65689.0), (8866, 65685.0), (8882, 65681.0), (8865, 65674.0), (8867, 65671.

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
 ('std_scaler', StandardScaler()),
 ])

df_scaled = pipeline.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=columns)

In [82]:
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = df_scaled[columns[:-1]]
y = df_scaled[columns[-1]]

# Perform train-test split with a 80-20 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (690, 2)
X_test shape: (173, 2)
y_train shape: (690,)
y_test shape: (173,)


In [136]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# tree_reg = DecisionTreeRegressor()
# tree_reg.fit(X_train, y_train)

regressor = RandomForestRegressor(
    n_estimators=100,
    max_leaf_nodes=16,
    n_jobs=-1,
    random_state=42
)
regressor.fit(X_train, y_train)

RandomForestRegressor(max_leaf_nodes=16, n_jobs=-1, random_state=42)

In [139]:
from sklearn.metrics import mean_squared_error

# y_pred = pd.DataFrame(tree_reg.predict(X_test))
y_pred = pd.DataFrame(regressor.predict(X_test))
y_pred = y_pred.rename(columns={'0':columns[-1]})

X_test_reset = X_test.reset_index(drop=True)

combined_org = pd.concat([X_test, y_test], axis=1)
combined_res = pd.concat([X_test_reset, y_pred], axis=1,ignore_index=True)
combined_res = combined_res.set_index(combined_org.index)
combined_res.columns = combined_org.columns

print(combined_org)
print(combined_res)

     Invoice Date  Vendor ID  Invoice Amount
715      0.188044   1.167199        1.627016
605     -0.994922   0.746764        1.622596
120      0.340210  -1.215267        0.463345
208      0.276399  -0.864905        0.390861
380      0.934148  -0.199216       -1.324560
..            ...        ...             ...
235      1.277748  -0.759796       -0.195896
333     -0.111379  -0.409433        0.155350
215      1.307199  -0.864905       -1.708048
796      1.164851   1.447489        1.230891
7        0.467833  -1.740811        1.123752

[173 rows x 3 columns]
     Invoice Date  Vendor ID  Invoice Amount
715      0.188044   1.167199       -0.095739
605     -0.994922   0.746764        0.043394
120      0.340210  -1.215267       -0.049990
208      0.276399  -0.864905       -0.245595
380      0.934148  -0.199216        0.058116
..            ...        ...             ...
235      1.277748  -0.759796       -0.037284
333     -0.111379  -0.409433       -0.077271
215      1.307199  -0.864905   

In [140]:
unscaled_pred = pipeline.inverse_transform(combined_res)
unscaled_org = pipeline.inverse_transform(combined_org)

unscaled_pred = pd.DataFrame(unscaled_pred, columns=columns)
unscaled_org = pd.DataFrame(unscaled_org ,columns=columns)

print(unscaled_org)
print(unscaled_pred)

tree_mse = mean_squared_error(unscaled_org['Invoice Amount'], unscaled_pred['Invoice Amount'])
tree_rmse = np.sqrt(tree_mse)
tree_rmse

     Invoice Date  Vendor ID  Invoice Amount
0          8565.0    65753.0         2469.78
1          8324.0    65741.0         2468.15
2          8596.0    65685.0         2040.65
3          8583.0    65695.0         2013.92
4          8717.0    65714.0         1381.32
..            ...        ...             ...
168        8787.0    65698.0         1797.54
169        8504.0    65708.0         1927.07
170        8793.0    65695.0         1239.90
171        8764.0    65761.0         2323.70
172        8622.0    65670.0         2284.19

[173 rows x 3 columns]
     Invoice Date  Vendor ID  Invoice Amount
0          8565.0    65753.0     1834.475198
1          8324.0    65741.0     1885.783666
2          8596.0    65685.0     1851.346226
3          8583.0    65695.0     1779.212622
4          8717.0    65714.0     1891.212817
..            ...        ...             ...
168        8787.0    65698.0     1856.031855
169        8504.0    65708.0     1841.285598
170        8793.0    65695.0   

375.57490009105675

In [155]:
print(final_vendors_df)

    Vendor ID  Invoice Date
0     65671.0          8867
1     65673.0          8870
2     65674.0          8865
3     65675.0          8865
4     65677.0          8863
5     65679.0          8867
6     65681.0          8867
7     65683.0          8865
8     65684.0          8867
9     65685.0          8866
10    65689.0          8866
11    65691.0          8868
12    65693.0          8867
13    65694.0          8861
14    65695.0          8868
15    65696.0          8868
16    65699.0          8869
17    65703.0          8869
18    65705.0          8867
19    65707.0          8864
20    65710.0          8863
21    65712.0          8865
22    65714.0          8867
23    65716.0          8865
24    65718.0          8865
25    65720.0          8864
26    65722.0          8868
27    65723.0          8866
28    65724.0          8867
29    65726.0          8870
30    65727.0          8865
31    65728.0          8867
32    65731.0          8866
33    65733.0          8870
34    65735.0       

In [174]:
# Final Prediction for Apr 2024

final_vendors_df_reorder = final_vendors_df[columns[:-1]]

input_pipeline = Pipeline([
    ('standardize', StandardScaler())
])

final_vendors_df_reorder_scaled = input_pipeline.fit_transform(final_vendors_df_reorder)
final_vendors_df_transformed = pd.DataFrame(final_vendors_df_reorder_scaled, columns=columns[:-1])

y_final = pd.DataFrame(regressor.predict(final_vendors_df_transformed))
final_unscaled_output = pd.concat([final_vendors_df_transformed, y_final], axis=1)

final_unscaled_output.columns = columns

#print(final_unscaled_output)

final_output_date_num = pd.DataFrame(pipeline.inverse_transform(final_unscaled_output) ,columns=columns)

#print(final_output_date_num)

final_output_date_num['Invoice Date'] = final_vendors_df_reorder['Invoice Date']
final_output_date_num['Vendor ID'] = final_vendors_df_reorder['Vendor ID']

#print(final_output_date_num)

final_output_date_num['Invoice Date'] = final_output_date_num['Invoice Date'].apply(date_from_num)
final_output = final_output_date_num
final_output['Invoice Amount'] = final_output['Invoice Amount'].round(2)
final_output['Vendor ID'] = final_output['Vendor ID'].astype(int)

final_check = set(final_output['Vendor ID'])
print(len(final_check))

final_output = final_output.sort_values(by=['Invoice Date', 'Vendor ID'], ascending=[True, True])

print(final_output)

52
   Invoice Date  Vendor ID  Invoice Amount
41  03-Apr-2024      65747         1873.48
13  04-Apr-2024      65694         1873.67
37  05-Apr-2024      65739         1926.10
4   06-Apr-2024      65677         1876.72
20  06-Apr-2024      65710         1925.82
19  07-Apr-2024      65707         1927.88
25  07-Apr-2024      65720         1934.38
34  07-Apr-2024      65735         1890.32
2   08-Apr-2024      65674         1756.24
3   08-Apr-2024      65675         1792.87
7   08-Apr-2024      65683         1810.96
21  08-Apr-2024      65712         1909.97
23  08-Apr-2024      65716         1908.21
24  08-Apr-2024      65718         1912.22
30  08-Apr-2024      65727         1917.64
38  08-Apr-2024      65740         1904.45
44  08-Apr-2024      65760         1818.85
9   09-Apr-2024      65685         1832.04
10  09-Apr-2024      65689         1840.41
27  09-Apr-2024      65723         1883.44
32  09-Apr-2024      65731         1886.21
40  09-Apr-2024      65744         1936.71
50  09-A